### About the dataset and clustering strategy
This dataset is about the neighborhoods in New York City. It consists of 5 boroughs and 306 neighborhoods.  
In order to segment the neighborhoods and explore them, I will use the latitude and longitude coordinates of each neighborhood.  
The dataset is publicly available from the NYU data repository: https://geo.nyu.edu/catalog/nyu_2451_34572

First, I will convert the addresses into their equivalent latitude and longitude values.   
Also, I will use the Foursquare API to explore the neighborhoods in New York City.  
Furthermore, I will use the Foursquare explore function to get the most common venue categories in each neighborhood.  
Afterwards, this feature will be used to group the neighborhoods into clusters.   
The k-means clustering algorithm will be used to complete this task.   
Finally, the Folium library will be used to visualize the neighborhoods in New York City and their emerging clusters.

### Downloading and exploring the data
The data is stored on the IBM Object storage and will be downloaded.  
The required Python libraries Pandas, Numpy, Geopy, Matplotlib, Folium, JSON and Scikit-learn will be imported.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')


!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 508.60 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 839.03 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.21 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  26.13 MB/s
vincent-0.4.4- 100% |###################

Let's parse the JSON string and cast it to a Python dictionary:

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a look at our data:

In [3]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

Notice how all the relevant data is stored in the features key, which is basically a list of the neighborhoods.  
Let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list:

In [5]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

This item seems to be about the neighborhood Wakefield in the borough Bronx.  
The relevant information is name, borough, and the coordinates.  
Let's put this information into a Pandas dataframe:

#### Transform the data into a *pandas* dataframe
First, let's define the column labels and instantiate the dataframe **neighborhoods**:

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Let's fill the neighborhoods dataframe by looping through the neighborhoods_data dictionary:

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's double-check that we got all neighborhoods and boroughs into the dataframe:

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Create a folium map of NYC and superimpose data points on top
The geopy geocode function will be quickly used get the central coordinates of NYC, so that we can spread the boroughs and neighborhoods evenly around it later:

In [11]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


Now, with the latitude and longitude coordinates of NYC, we can create a Folium map of NYC and superimpose the respective neighborhoods and boroughs on top:

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Neighborhoods of Manhattan
Let's repeat the same steps and take a closer look at the Manhattan borough only:

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### Use Foursquare API to explore and segment neighborhoods
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.  
Let's start off with defining the Foursquare credentials and the version:

In [17]:
CLIENT_ID = 'YMZZBJXVZPYIDLPZNUPSZTUQVNFOWKBCT1L41VBWULO1LFAH' # your Foursquare ID
CLIENT_SECRET = 'QSRA3QGSIGA5CDS05EPOXDAR1NEZTLQCFMU3PSSZIGJPG1KW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's take a look at the first neighboorhood in our Manhattan dataframe and get its coordinates:

In [18]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [19]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


Now, let's use the Foursquare API to get the top 100 venues that are in Marble Hill within a radius of 500 meters from its center coordinates:

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results # display results


{'meta': {'code': 200, 'requestId': '5bed33b61ed2195a71ac7bb1'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

Apparently, there are only 22 venues in a radius of 500 meters of Marble Hill center.  
Let's see what kind of venues have been found by casting the JSON to a Pandas dataframe:

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON data into a Pandas dataframe

# filter columns 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin' Donuts,Donut Shop,40.876993,-73.906507


Double-check whether we got all 22 venues:

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


### Exploring Neighborhoods in Manhattan
After exploring our data, let's create a function to do the same process to all neighborhoods in Manhattan:

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

With the function complete, let's create a new dataframe with all the neighborhoods in Manhattan:

In [29]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the dataframe and the top 5 rows:

In [30]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3302, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


Let's see how many venues have been returned for each neighborhood given the radius of 500 meters:

In [32]:
manhattan_venues.groupby('Neighborhood')['Neighborhood'].count()

Neighborhood
Battery Park City       98
Carnegie Hill          100
Central Harlem          47
Chelsea                100
Chinatown              100
Civic Center           100
Clinton                100
East Harlem             43
East Village           100
Financial District     100
Flatiron               100
Gramercy               100
Greenwich Village      100
Hamilton Heights        59
Hudson Yards            59
Inwood                  55
Lenox Hill             100
Lincoln Square         100
Little Italy           100
Lower East Side         60
Manhattan Valley        63
Manhattanville          40
Marble Hill             22
Midtown                100
Midtown South          100
Morningside Heights     48
Murray Hill            100
Noho                   100
Roosevelt Island        24
Soho                   100
Stuyvesant Town         19
Sutton Place           100
Tribeca                100
Tudor City              79
Turtle Bay             100
Upper East Side        100
Upper West Side

Most neighborhoods returned 100 or more venues. Given the density of Manhattan, this is not surprising.  
Let's see how many unique venue categories have been collected in this dataframe:

In [33]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 329 uniques categories.


### Analyze each neighborhood
Now, each each venue in a neighborhood will be sorted into its respective category.  
Afterwards, the mean frequency of occurence of a respective category in a neighborhood will be displayed.  
In essence, let's find out what kind of venues are in each neighborhood and how frequent these category of venues are in regards to the total of venues.  
For example, if there are 5 restaurants with the category **Italian restaurant** in our collection of 100 venues, the category **Italian restaurant** will be assigned the value 0.05.

First, let's use one-hot encoding to convert the categorical venue category to a binary classifier column for each venue category:

In [34]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,S

Let's see whether we all 329 categories and the neighborhood column are in the dataframe:

In [35]:
manhattan_onehot.shape

(3302, 330)

Good.  
Let's group the rows by neighborhood and by taking the mean of the frequency of occurrence of each category as discussed in our strategy:

In [37]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,S

Let's confirm whether we have all 40 neighborhoods of Manhattan and all venue categories in our dataframe:

In [38]:
manhattan_grouped.shape

(40, 330)

Let's find out what the top 5 most common venues for each neighborhood in Manhattan are:

In [39]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2               Hotel  0.05
3  Italian Restaurant  0.03
4                 Gym  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1     Coffee Shop  0.05
2            Café  0.04
3  Cosmetics Shop  0.04
4     Yoga Studio  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.06
1  Fried Chicken Joint  0.04
2    French Restaurant  0.04
3   Chinese Restaurant  0.04
4       Cosmetics Shop  0.04


----Chelsea----
                 venue  freq
0   Italian Restaurant  0.06
1          Coffee Shop  0.06
2       Ice Cream Shop  0.04
3            Nightclub  0.04
4  American Restaurant  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1           Cocktail Bar  0.04
2  Vietnamese Restaurant  0.04
3    American Restaurant  0.04
4     Dim Sum Restaurant  0.04


----Civic Center----
                  

Great.  
Let's put this knowledge into a new dataframe with each neighborhood and the top 10 most common venues.  
First, let's write a function to sort the venues in descending order:

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Gym,Sandwich Place,Clothing Store,Boat or Ferry,Fountain,Plaza
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore,Bar
2,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,Chinese Restaurant,American Restaurant,French Restaurant,Fried Chicken Joint,Boutique,Beer Bar,Library
3,Chelsea,Italian Restaurant,Coffee Shop,Ice Cream Shop,Nightclub,American Restaurant,Seafood Restaurant,Theater,Hotel,Bakery,Art Gallery
4,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Bakery,Noodle House,Salon / Barbershop
5,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,Cocktail Bar,Sushi Restaurant,Coffee Shop,French Restaurant,Gym,Park,Sandwich Place
6,Clinton,Theater,Italian Restaurant,Hotel,American Restaurant,Coffee Shop,Gym / Fitness Center,Wine Shop,Spa,Gym,Indie Theater
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Pizza Place,Steakhouse,Street Art,French Restaurant,Café
8,East Village,Bar,Ice Cream Shop,Wine Bar,Cocktail Bar,Mexican Restaurant,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Speakeasy
9,Financial District,Coffee Shop,Hotel,Steakhouse,Food Truck,Pizza Place,Park,Japanese Restaurant,Gym,Italian Restaurant,Bar


### Cluster the neighborhoods in Manhattan
Let's cluster the neighborhoods in 5 cluster.  
The k-means algorithm will be used for this:

In [42]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

On first guessing, it looks like we found 2 main clusters and 3 outliers.  
Let's create a new dataframe that includes the clusters, as well as the top 10 venues for each neighborhood.

In [43]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Coffee Shop,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place,Pizza Place
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Bakery,Noodle House,Salon / Barbershop
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Pizza Place,Sandwich Place,Mexican Restaurant,Supermarket,Deli / Bodega,Grocery Store,Clothing Store,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Mexican Restaurant,Lounge,Café,Pizza Place,Restaurant,Deli / Bodega,Bakery,Wine Bar,American Restaurant,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sandwich Place,Yoga Studio,Chinese Restaurant


Finally, let's visualize the resulting clusters:

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters
Finally, let's examine the cluster and find out why the neighborhoods have been clustered as such.  
First, let's take a quick look at the outliers:

In [48]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Thai Restaurant,Pizza Place,Steakhouse,Street Art,French Restaurant,Café
13,Lincoln Square,Gym / Fitness Center,Theater,Italian Restaurant,Concert Hall,Plaza,French Restaurant,Café,Bakery,Indie Movie Theater,Opera House


In [49]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore,Bar


In [50]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Little Italy,Bakery,Chinese Restaurant,Café,Yoga Studio,Ice Cream Shop,Sandwich Place,Salon / Barbershop,Pizza Place,Seafood Restaurant,Cocktail Bar


Now, let's look at the 2 main clusters and try to profile them:

In [51]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Comfort Food Restaurant,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant,Sandwich Place,Pizza Place
3,Inwood,Mexican Restaurant,Lounge,Café,Pizza Place,Restaurant,Deli / Bodega,Bakery,Wine Bar,American Restaurant,Park
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sandwich Place,Yoga Studio,Chinese Restaurant
5,Manhattanville,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Pharmacy,Supermarket,Burger Joint,Spanish Restaurant,Bus Station,Check Cashing Service,Chinese Restaurant
6,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,Chinese Restaurant,American Restaurant,French Restaurant,Fried Chicken Joint,Boutique,Beer Bar,Library
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Juice Bar,Boutique,French Restaurant,Gym / Fitness Center,Hotel
9,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Pizza Place,Japanese Restaurant,Deli / Bodega,Ice Cream Shop,Sushi Restaurant,Wine Shop
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Burger Joint,Gym,Wine Shop,Steakhouse,Sporting Goods Shop
11,Roosevelt Island,Park,Sandwich Place,School,Supermarket,Dry Cleaner,Baseball Field,Greek Restaurant,Liquor Store,Gym,Coffee Shop
12,Upper West Side,Italian Restaurant,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Wine Bar,Indian Restaurant,Bakery,Pub,Sushi Restaurant,Mexican Restaurant


Here, in cluster 1 (purple on the map), we can clearly see that the most common or second most common category is **Italian restaurant**.  
Also, coffee shops, parks, gyms and other restaurants are prevalent.  
Considering their placing, spread out in the center of Manhattan, we could profile this cluster as FOOD AND LEISURE.  
In essence, neighborhoods that people visit after work to enjoy themselves.

Let's take a look at the second main cluster in the South of Manhattan:

In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Bakery,Noodle House,Salon / Barbershop
2,Washington Heights,Café,Bakery,Pizza Place,Sandwich Place,Mexican Restaurant,Supermarket,Deli / Bodega,Grocery Store,Clothing Store,Chinese Restaurant
15,Midtown,Hotel,Coffee Shop,Theater,Food Truck,Steakhouse,Sporting Goods Shop,Bakery,Sandwich Place,Spa,Cocktail Bar
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,French Restaurant,Chinese Restaurant,Seafood Restaurant,Indian Restaurant,Boutique,Café,Gourmet Shop
19,East Village,Bar,Ice Cream Shop,Wine Bar,Cocktail Bar,Mexican Restaurant,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Speakeasy
20,Lower East Side,Coffee Shop,Art Gallery,Café,Shoe Store,Bakery,Cocktail Bar,Pizza Place,Chinese Restaurant,Ramen Restaurant,Sandwich Place
25,Manhattan Valley,Pizza Place,Bar,Coffee Shop,Indian Restaurant,Italian Restaurant,American Restaurant,Playground,Café,French Restaurant,Mexican Restaurant
28,Battery Park City,Coffee Shop,Park,Hotel,Italian Restaurant,Gym,Sandwich Place,Clothing Store,Boat or Ferry,Fountain,Plaza
33,Midtown South,Korean Restaurant,Coffee Shop,Japanese Restaurant,Hotel Bar,Italian Restaurant,Gym / Fitness Center,Boutique,Cocktail Bar,Cosmetics Shop,Hotel
37,Stuyvesant Town,Bar,Park,Rental Car Location,Gas Station,Baseball Field,Pet Service,Cocktail Bar,Harbor / Marina,Coffee Shop,German Restaurant


In this cluster, bars and coffee shops seem to be the most prevalent.  
Considering their placing in the most expensive area of Manhattan, this cluster could represent BUSINESS MEETINGS, QUICK DINING AND DATING.